# Lab 1.4 Tool Calling Agent

The tool-calling ability of LLMs refers to their capability to interact with external systems, APIs, or tools during their execution to perform specific tasks or augment their responses with functionalities beyond their internal knowledge. In this notebook we will look at how to use a simple tool with LLMs

## Models that support tool calling


In general, openAI models (e.g. `gpt-4o-mini`) as well as certain LLM NIMs (e.g. `llama-3.1-405b-instruct`) support tool calling. To find out which LLM NIM support tool calling, we can use the function below: 

In [1]:
import dotenv

# requires NVIDIA_API_KEY in .env
dotenv.load_dotenv()

True

In [2]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
tool_models = [
    model for model in ChatNVIDIA.get_available_models() if model.supports_tools
]
tool_models

/home/ubuntu/miniconda3/envs/workshop/lib/python3.12/site-packages/langchain_nvidia_ai_endpoints/_common.py:176: UserWarning: An API key is required for the hosted NIM. This will become an error in the future.
  warnings.warn(


[Model(id='nv-mistralai/mistral-nemo-12b-instruct', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=None, supports_tools=True, supports_structured_output=True, base_model=None),
 Model(id='meta/llama-3.1-8b-instruct', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=None, supports_tools=True, supports_structured_output=True, base_model=None),
 Model(id='meta/llama-3.1-70b-instruct', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=None, supports_tools=True, supports_structured_output=True, base_model=None),
 Model(id='meta/llama-3.1-405b-instruct', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=None, supports_tools=True, supports_structured_output=True, base_model=None),
 Model(id='meta/llama-3.2-3b-instruct', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=None, supports_tools=True, supports_structured_output=True, base_model=None),
 Model(id='mistralai/mistral-large-2-instruct', model_type='chat', client='ChatN

## A basic example of tool-calling agent in LangGraph

Now we can look at a basic example of tool calling agent in LangGraph. 

### Define the model

#### Option 1: LLM NIM

In [3]:
import os
from langchain_nvidia_ai_endpoints import ChatNVIDIA

# initialize the LLM client
model = ChatNVIDIA(
  model="meta/llama-3.2-3b-instruct",
  api_key=os.getenv("NVIDIA_NIM_API_KEY"), 
  temperature=0.2,
  top_p=0.7,
  max_tokens=1024,
)

#### Option 2: OpenAI model (try it outside the workshop)

> OpenAI model requires an API key. This is not covered in the workshop, but we provide snippets so that you can try it out on your own. 


Go to `2025-01-biologic-summit-workshop/.env` file, and uncomment this line: 

```bash
OPENAI_API_KEY="YOUR OPENAI API KEY HERE"
```
Put your openAI API key, so that it can loaded into environment variable. Then run the following code: 

In [4]:
# from langchain_core.tools import tool
# from langchain_openai import ChatOpenAI
# model = ChatOpenAI(model="gpt-4o")

### Define a tool

In [5]:
from langchain_core.tools import tool

@tool
def magic_function_1(input: int) -> int:
    """Applies a magic function 1 to an input."""
    return input * 100

@tool
def magic_function_2(input: int) -> int:
    """Applies a magic function 2 to an input."""
    return input/100


### Add the tool to a list

In [6]:
# you can 
tools = [magic_function_1, magic_function_2]

### Add system prompt

In [7]:
system_message = "You are a helpful assistant. Repsond truthfully."

### Add memory

In [8]:
from langgraph.checkpoint.memory import MemorySaver

# create session_id
session_id = "session_1"

# create a memory saver
memory = MemorySaver()

### Built an agent

In [9]:
from langgraph.prebuilt import create_react_agent

langgraph_agent_executor = create_react_agent(
    model, tools, state_modifier=system_message, checkpointer=memory
)


### Try it out

In the conversation the user asks a qeustion first, then the agent respond. This counts as 1 conversation turn. For example: 

In [10]:
query = "What is the result of magic function with input of 2"
messages = langgraph_agent_executor.invoke(
    {"messages": [("human", query)]}, 
    config= {"configurable": {"thread_id": session_id}}
)

print(messages["messages"][-1].content)

The result of the magic function with an input of 2 is 200.


Now if the user asks another question, this counts as the 2nd turn. This is a good time to check the memory, to see if the agent has remembered the previous conversation. 

In [11]:

# 2nd turn
query = "What's the question I just asked?"
messages = langgraph_agent_executor.invoke(
    {"messages": [("human", query)]}, 
    config= {"configurable": {"thread_id": session_id}}
)
print(messages["messages"][-1].content)

You asked: "What is the result of magic function with input of 2"


Now let's test to see if the agent can differentiate magic function 1 and 2: 

In [12]:
# 3rd turn
query = "What's the result of magic function 2 with input of 2?"
messages = langgraph_agent_executor.invoke(
    {"messages": [("human", query)]}, 
    config= {"configurable": {"thread_id": session_id}}
)
print(messages["messages"][-1].content)

The result of the magic function 2 with an input of 2 is 0.02.


In here you can see that the agent has remembered the previous conversation, and correctly applied the correct tool. 

Please proceed to Lab 1.5. 